<div class="alert alert-success">
    <h1>Assignment 1: Linear Regression</h1>
    <h3 align='center'>Machine Learning Course (Fall 2018)</h3>
    <h5 align='center'>Seyed Naser RAZAVI</h5>
</div>

Note that the contents in this assigment and the following assignments are heavily inspired from the amazing machine learning course presented at Stanford by Andrew Ng.

However, here in this series assignments for our machine learning course, we have used **Python** language which we think is much more suitable than Matlab for artificial intelligence students. Also we have changed and modified some parts of the assignments and the related descriptions for those parts.

## Linear Regression

First, let us import two useful python library which we will use through this assinment.
- The first library is `numpy` to work with N-dimensional arrays.
- The second library is `matplotlib` for plotting.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# setup for printing numpy arrays
np.set_printoptions(precision=2)

## Part 0: Warm up!

In this part, you will write a very simple function in python which creates and prints a 5-by-5 identity matrix. Complete the definition of the following function, by adding `A = np.eye(5)` to it.

In [ ]:
def warm_up():
    ### your code here ###
    print(A)

Now that you have defined this function, you can call it by executing the following cell. After running this cell, you should see something like this:
```python
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
```

Recall: To run a cell, you can use `<Shift> + <Enter>` or `<Ctrl> + <Enter>`. 

In [ ]:
warm_up()

## Part 1: Linear regression with one variable

In this part, your task is to implement linear regression with one variable to predict profits for a food truck.

Suppose you are the CEO of a restaurant franchise and are considering different cities for opening a new outlet. The chain already has trucks in various cities and you have data for profits and populations from the cities. You would like to use this data to help you select which city to expand to next.

<img src='imgs/restaurant_dataset.png' width='50%'/>

The file `data/ex1data1.txt` contains the dataset for our linear regression problem. The first column is the `population` of a city and the second column is the `profit` of a food truck in that city. A negative value for profit indicates a loss.

### Loading data

The function `load_data()` has already been set up to load this data for you. It recieves the file name which contains the data (in this case `data/ex1data1.txt`) and returns `X` and `y` as 2D matrices.

In [ ]:
def load_data(fname='data/ex1data1.txt'):
    data = np.genfromtxt(fname, delimiter=',')
    X = data[:, 0].reshape(-1, 1)  # Get the 1st column and convert it to a 2d matrix
    y = data[:, 1].reshape(-1, 1)  # Get the 2nd column and convert it to a 2d matrix
    return X, y

In [ ]:
X, y = load_data(fname='data/ex1data1.txt')

print(X.shape)
print(y.shape)

A more general implementation of this function is given here. This implementation works for any number of columns in the text file which are separated by commas (This is useful for the next part of this assignment, when we deal with linear regression with multiple variables). 

Here we assume that the first columns of the text file are the input features and the last column is the target value which we are going to predict. Be sure that you understand how we have separated the columns and stored them in `X` and `y`.

In [ ]:
def load_data(fname='data/ex1data1.txt'):
    data = np.genfromtxt(fname, delimiter=',')
    n = data.shape[1] - 1            # number of features
    X = data[:, :-1].reshape(-1, n)  # Get all but the last column and convert to a 2d array
    y = data[:,  -1].reshape(-1, 1)  # Get the last column and convert to a 2d array
    return X, y

In [ ]:
X, y = load_data(fname='data/ex1data1.txt')

print(X.shape)
print(y.shape)

### Plotting data

Before starting on any task, it is often useful to understand the data by visualizing it. For this dataset, you can use a scatter plot to visualize the data, since it has only two properties to plot (`profit` and `population`). (Many other problems that you will encounter in real life are multi-dimensional and can’t be plotted on a 2-d plot.) 

In [ ]:
def plot_data(X, y):
    plt.figure(figsize=(12, 8), dpi=120)
    plt.scatter(X, y, s=20, c='r', marker='x', alpha=0.5)
    plt.xlabel('Population of city in 10,000')
    plt.ylabel('Profit in $10,000s')   
    plt.title('Profit vs. Population')

Now, you can call `plot_data` to create a scatter plot of the data. Run the following cell to see the data.

In [ ]:
plot_data(X, y)

To learn more about the `scatter` command, you can type `help(plt.scatter)` in a new cell or to search online for plotting documentation. For example, to change the markers to red “x”, we used the options `c='r'` and `marker='x'` together with the scatter command.

In [ ]:
help(plt.scatter)

### Gradient descent

In this part, you will ﬁt the linear regression parameters $\theta$ to our dataset using gradient descent.


#### Update Equations

The objective of linear regression is to minimize the **cost function** (here, MSE)
$$J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})^2$$
where the **hypothesis** $h_\theta(x)$ is given by the linear model
$$h_\theta(x) = \theta^T x = \theta_0 + \theta_1 x$$


Recall that the parameters of your model are the $\theta_j$ values. These are the values you will adjust to minimize cost $J(\theta)$. One way to do this is to use the **full-batch gradient descent** algorithm. In full-batch gradient descent, each iteration performs the update
$$\theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_{j}^{(i)}$$
With each step of gradient descent, your parameters $\theta_j$ come closer to the optimal values that will achieve the lowest cost $J(\theta)$.


<div class="alert alert-success">
    <strong>Implementation Note:</strong> We store each example as a row in the the $X$ matrix in python. To take into account the intercept term ($\theta_0$), we add an additional first column to $X$ and set it to all ones. This allows us to treat $\theta_0$ as simply another *feature*.
</div>

### Part 1-1: Implementation

#### Hypothesis function

First, let us implement the hypothesis function in linear regression. The following function `h` gets the input data `X` and the parameters as input and outputs predictions by a simple matrix multiplication. Note that the result is a 2d matrix (m-by-1).

In [ ]:
def h(X, theta):
    """ Hypothesis function for linear regression.
    
    Arguments:
        - X: Input matrix, a 2d array of shape (m, n+1)
        - theta: parameters, a 2d array of shape (n+1, 1)
        
    Output:
        - Predictions for input data (m-by-1)
    """
    return X @ theta

 In the following lines, we add another dimension to our data to accommodate the $\theta_0$ intercept term. We also initialize the initial parameters to 0 and the learning rate $\alpha$ to 0.01.

In [ ]:
X = np.c_[np.ones((X.shape[0], 1)), X]  # concatinating a column of 1s
theta = np.zeros((2, 1))                # initialize parameters to 0
alpha = 0.01                            # learning rate
num_iterations = 1500                   # number of iterations

Note that in the above code, the first satatement is used to add a column of ones to the left of the input matrix `X`. This is another way to conacatinate matrices in `numpy`. Of course, you could use the `np.concatinate()` or the `np.hstack()` which we saw in the class. Here, I have used `np.c_[]`, because it's a common way for matrix concatination in `numpy` and you will see it a lot in the codes which are available online.

#### Computing the cost function

As you perform gradient descent to learn minimize the cost function J(θ), it is helpful to monitor the convergence by computing the cost. In this section, you will implement a function to calculate J(θ) so you can check the convergence of your gradient descent implementation. 

Your next task is to complete the code in the `compute_cost()` function, which is a function that computes J(θ). As you are doing this, remember that the variables `X` and `y` are not scalar values, but matrices whose rows represent the examples from the training set. 

<div class="alert alert-danger">
    <strong>Imortant Note:</strong> Notice that your implementation must be at most 3 or 4 lines of code and you are not allowed to use loops anywhere in your code. Instead, you should rely on vectorized computations that *numpy* provides for you.
        </ul>
</div>

Note: Because the function is not implemented yet, we have included an exception statement which states "You should implement this function.". To implement the function, you should remove this line and replace it by your code.

In [ ]:
def compute_cost(X, y, theta):
    """ Vectorized implementation of MSE cost function.
    
    Arguments:
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m, 1)
        - theta: Parameters, a 2d array of shape (n + 1, 1)
        
    Outputs:
        - the mean squared error value
    """
    raise NotImplementedError('You should implement compute_cost function')

Once you have completed the function, the next step will run `compute_cost()` once using θ initialized to zeros, and you will see the cost printed to the screen. You should expect to see a cost of `32.07`.

In [ ]:
print('J([0, 0]) = %.2f' % compute_cost(X, y, theta))

#### Gradient descent

Now, it is time to implement gradient descent algorithm. An unvectorized implementation of full-batch gradient descent algorithm is already written for you. The unvectorized implementation is correct and works fine for the purpose of this assignment. 

But keep in mind that always use a **vectorized implementation** for real world problems, because the vectorized implementation is *much faster* and also *much simpler* comparing to the unvectorized implementation.

In [ ]:
def gradient_descent_unvectorized(X, y, theta, alpha, num_iterations):
    """ Unvectorized implementation of full-batch gradient descent.
    
    Arguments:
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m, 1)
        - theta: Initial guess for parameters, a 2d array of shape (n + 1, 1)
        - alpha: Learning rate (or step size) in G.D. algorithm
        - num_iterations: Maximum number of iterations in G.D. algorithm
        
    Outputs:
        - theta: the final value of parameters after learning
    """
    m, n = X.shape
    
    for iteration in range(num_iterations):
        
        # compute cost for current value of paremetrs
        cost = compute_cost(X, y, theta)
        
        # print cost every 100 iterations
        if iteration % 100 == 0:
            print('%4d: J(%s) = %.2f' %(iteration, theta.ravel(), cost))
        
        # compute gradients
        grads = np.zeros_like(theta)
        
        for j in range(n):
            for i in range(m):
                term = h(X[i], theta) - y[i]
                grads[j] += X[i, j] * term
        
        # update parameters
        for j in range(n):
            theta[j] = theta[j] - (alpha / m) * grads[j]  
            
    return theta

Now, let us run the unvectorized implementation of the full-batch gradient descent algorithm to see some results. Run the following cell to see the results.

In [ ]:
# load data
X, y = load_data('data/ex1data1.txt')

# add the column of ones to the X
X = np.c_[np.ones((X.shape[0], 1)), X]

# run gradient descent algorithm
theta = np.random.randn(2, 1)
alpha = 0.01
num_iterations = 2001
theta = gradient_descent_unvectorized(X, y, theta, alpha, num_iterations)

<div class="alert alert-success">
    <strong>Verification of Gradient Descent: </strong>  A good way to verify that gradient descent is working correctly is to look at the value of J(θ) and check that it is decreasing with each step. Assuming you have implemented the compute_cost() function correctly, your value of J(θ) should never increase, and should converge to a steady value by the end of the algorithm.
</div>

#### Gradient descent: vectorized implementation

Now, your task is to implement the vectorized version of the full-batch gradient descent algorithm by inspiring the unvectorized version we provided for you. Complete the following function named `gradient_descent()` to finish the task. Not that a correct implementation must be 5 or 6 lines of code at most.

As you program, make sure you understand what you are trying to optimize and what is being updated. Keep in mind that the cost J(θ) is parameterized by the vector θ, not X and y. That is, we minimize the value of J(θ) by changing the values of the vector θ, not by changing X or y. Refer to the equations in this handout and to the video lectures if you are uncertain. 

In [ ]:
def gradient_descent(X, y, theta, alpha, num_iterations):
    """ A vectorized implementation of full-batch gradient descent.
    
    Arguments:
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m, 1)
        - theta: Initial guess for parameters, a 2d array of shape (n + 1, 1)
        - alpha: Learning rate (or step size) in G.D. algorithm
        - num_iterations: Maximum number of iterations in G.D. algorithm
        
    Outputs:
        - theta: the final value of parameters after learning
    """
    raise NotImplementedError('You should implement gradient_descent function.')

After you are finished, use your final parameters to plot the linear fit. The result should look something like this:

<img src='imgs/linear_regression_result_for_restaurant_dataset.png' width='70%'>

Your ﬁnal values for θ will also be used to make predictions on profits in areas of 35,000 and 70,000 people. Note the way that the following lines uses matrix multiplication, rather than explicit summation or looping, to calculate the predictions. This is an example of **code vectorization** in Python.


In [ ]:
def part2_1():
    # load and prepare data
    X, y = load_data('data/ex1data1.txt')
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    # initialization
    theta = np.zeros((2, 1))
    alpha = 0.01
    num_iterations = 1500
    
    # print cost for initial value of parameters
    cost = compute_cost(X, y, theta)  # should be 32.07
    print("Cost = %.2f" % cost)
    
    # run gradient descent
    theta = gradient_descent(X, y, theta, alpha, num_iterations)
    
    # print the final value of parameters after running G.D.
    print("theta = %s" % theta.ravel())
    
    # predict for two different inputs
    prediction1 = np.array([1, 3.5]).dot(theta)
    prediction2 = np.array([1, 7.0]).dot(theta)
    print("Prediction for 3.5 is: %.2f" % prediction1)
    print("Prediction for 7.0 is: %.2f" % prediction2)
    
    # plot data and the fit
    plot_data(X[:, 1], y)
    plt.plot(X[:, 1], X @ theta, 'k-')
    plt.show()

In [ ]:
part2_1()

<div class="alert alert-danger">
    <strong>Debugging:</strong> Here are some things to keep in mind as you implement gradient descent:
        <ul>
            <li>Python array indices start from zero, not one. If you’re storing θ0 and θ1 in a vector called theta, the values will be theta[0] and theta[1].</li>
            <li>If you are seeing many errors at runtime, inspect your matrix operations to make sure that you’re adding and multiplying matrices of compatible dimensions. Printing the dimensions of variables with the *shape* command will help you debug.</li>
            <li>By default, Python interprets math operators to be element-wise operators. This is a common source of size incompatibility errors. If you want matrix multiplication, you need to use the `@` operator to specify this to Python. For example, `A * B` does an element-wise matrix multiplication, while `A @ B` does a normal matrix multiplication.</li>
        </ul>
</div>

### Part 1-2: Visualizing $J(\theta)$

To understand the cost function J(θ) better, you will now plot the cost over a 2-dimensional grid of θ0 and θ1 values. You will not need to code anything new for this part, but you should understand how the code you have written already is creating these images. In the next step, there is code set up to calculate J(θ) over a grid of values using the `compute_cost()` function that you wrote.

In [ ]:
def part1_2():
    # load and prepare data
    X, y = load_data("data/ex1data1.txt")
    X = np.c_[np.ones((X.shape[0], 1)), X]

    # compute cost function for a range of different values for parameters
    theta0_vals = np.linspace(-20, 20, 100)
    theta1_vals = np.linspace(-6, 6, 100)

    J_vals = np.zeros((len(theta0_vals), len(theta1_vals)), dtype=np.float64)
    for i, v0 in enumerate(theta0_vals):
        for j, v1 in enumerate(theta1_vals):
            theta = np.array((v0, v1)).reshape(2, 1)
            J_vals[i, j] = compute_cost(X, y, theta)

    T0, T1 = np.meshgrid(theta0_vals, theta1_vals)

    # plot cost function
    fig = plt.figure(figsize=(8, 6))
    ax = fig.gca(projection='3d')
    ax.plot_surface(T0, T1, J_vals, cmap=plt.cm.rainbow)
    plt.show()

    # run gradient descent to find optimal theta
    theta = gradient_descent(X, y, theta, alpha=0.02, num_iterations=1500)

    # plot contours of cost function and the final value of parameters
    fig = plt.figure(figsize=(8, 6))
    plt.contour(T0, T1, J_vals.T, np.logspace(-2, 3, 20), cmap=plt.cm.rainbow)
    plt.plot(theta[0], theta[1], 'rx', markersize=10)
    plt.show()

In [ ]:
part1_2()

The purpose of these graphs is to show you that how J(θ) varies with changes in θ0 and θ1. The cost function J(θ) is bowl-shaped and has a global mininum. (This is easier to see in the contour plot than in the 3D surface plot). This minimum is the optimal point for θ0 and θ1, and each step of gradient descent moves closer to this point.

## Part 2: Multi-variable Linear Regression

In this part, you will implement linear regression with multiple variables to predict the prices of houses. Suppose you are selling your house and you want to know what a good market price would be. One way to do this is to first collect information on recent houses sold and make a model of housing prices.

### Dataset

The file `data/ex1data2.txt` contains a training set of housing prices in Portland, Oregon. The first column is the size of the house (in square feet), the second column is the number of bedrooms, and the third column is the price of the house.

<img src='imgs/house_dataset.png' width='50%'>

### Part 2-1: Feature normalization

By looking at the dataset for this part, note that house sizes are about 1000 times the number of bedrooms. When features differ by orders of magnitude, first performing feature scaling can make gradient descent converge much more quickly.

Your task here is to complete the code in `feature_normalize()` function to:
- Subtract the **mean** value of each feature from the dataset.
- After subtracting the mean, additionally scale (divide) the feature values by their respective **standard deviations**.

In [ ]:
def feature_normalize(X):
    """A vectorized implementation of feature normalization.
    
    Arguments:
        - X: The raw input features, a 2d array of shape (m, n)
        
    Outputs:
        - X_norm: The normalized version of the input matrix
        - mu: The mean of the features
        - sigma: The std dev of the features 
    """
    raise NotImplementedError("You should implement feature_normalize function.")

The standard deviation is a way of measuring how much variation there is in the range of values of a particular feature (most data points will lie within ±2 standard deviations of the mean); this is an alternative to taking the range of values (max-min). 

In Python, you can use the `std` function from `numpy` to compute the standard deviation and the `mean` function to compute the mean. For example, inside `feature_normalize()`, the quantity `X[:, 0]` contains all the values of x1 (house sizes) in the training set, so `np.std(X[:, 0])` computes the standard deviation of the house sizes. 

**Note:** At the time that `feature_normalize()` is called, the extra column of 1’s corresponding to x0 = 1 has not yet been added to X. 

You will do this for all the features and your code should work with datasets of all sizes (any number of features / examples). Note that each column of the matrix X corresponds to one feature. 

<div class="alert alert-success">
    <strong>Implementation Note:</strong> When normalizing the features, it is important to store the values used for normalization - the *mean* value and the *standard deviation* used for the computations. After learning the parameters from the model, we often want to predict the prices of houses we have not seen before. Given a new x value (living room area and number of bedrooms), we must first normalize x using the mean and standard deviation that we had previously computed from the training set.
</div>

After you finished with writting the `feature_normalize()` function, you can run the following two cells to see the noramalized features (only for first five inputs) and the mean value and the standard deviation for the two input features.

In [ ]:
def part2_1():
    
    # load data as 2d matrices
    X, y = load_data('data/ex1data2.txt')

    # feature normalization
    X_norm, mu, sigma = feature_normalize(X)
    
    # display the results
    print('Matrix X after normalization:')
    print(X_norm[:5, :])
    
    print("mean = %s " % mu)
    print("std dev = %s" % sigma)

In [ ]:
part2_1()

### Part 2-2: Gradient Descent

Previously, you implemented gradient descent on a univariate regression problem. The only diﬀerence now is that there is one more feature in the matrix X. The hypothesis function and the batch gradient descent update rule remain unchanged. 

You should complete the code in `compute_cost_multi()` and `gradient_descent_multi()` to implement the cost function and gradient descent for linear regression with multiple variables. If your code in the previous part (single variable) already supports multiple variables, you can use it here too. 

Make sure your code supports any number of features and is well-vectorized. You can use `X.shape[1]` to ﬁnd out how many features are present in the dataset.


In [ ]:
def compute_cost_multi(X, y, theta):
    """ A vectorized implementation of MSE cost function
    
    Arguments:
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m, 1)
        - theta: Parameters, a 2d array of shape (n + 1, 1)
        
    Outputs:
        - the mean squared error value
    """
    raise NotImplementedError('You should implement compute_cost_multi function.')

In [ ]:
def gradient_descent_multi(X, y, theta, alpha, num_iterations):
    """ A vectorized implementation of full-batch gradient descent.
    
    Arguments:
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m, 1)
        - theta: Initial guess for parameters, a 2d array of shape (n + 1, 1)
        - alpha: Learning rate (or step size) in G.D. algorithm
        - num_iterations: Maximum number of iterations in G.D. algorithm
        
    Outputs:
        - J_history: a list of computed costs during G.D. algorithm
        - theta: the final value of parameters after learning
    """
    raise NotImplementedError('You should implement gradient_descent_multi function.')

<div class="alert alert-success">
    <strong>Implementation Note:</strong>  In the multivariate case, the cost function can also be written in the following vectorized form:
    $$J(\theta)=\frac{1}{2m}(X \theta - y)^T (X \theta - y)$$
    
    The vectorized version is efficient when you’re working with numerical computing tools like NumPy. If you are an expert with matrix operations, you can prove to yourself that the two forms are equivalent.

</div>

Run the following two cells to see the result of your implementation.

In [ ]:
def part2_2():
    
    # load data
    X, y = load_data('data/ex1data2.txt')
    
    # normalize features
    X, mu, sigma = feature_normalize(X)
    
    # add a column of ones to X
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    # run gradient descent algorithm
    num_iterations = 400
    alpha = 0.01
    theta = np.random.randn(3, 1)
    J_history, theta = gradient_descent_multi(X, y, theta, alpha, num_iterations)
    
    # print the final value of parameters
    print('theta = %s' % theta.ravel())
    
    # plot cost versus iteration
    plt.plot(J_history)
    plt.title("Alpha = %f" % (alpha))
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost J')
    plt.show()

In [ ]:
part2_2()

### Part 2-3: Selecting learning rate

In this part of the exercise, you will get to try out diﬀerent learning rates for the dataset and find a learning rate that converges quickly. You can change the learning rate by modifying and changing the part of the code that sets the learning rate. 

The next phase will call your `gradient_descent()` function and run gradient descent for about 50 iterations at the chosen learning rate. The function should also return the history of J(θ) values in a vector J. After the last iteration, the script plots the J values against the number of the iterations. 

If you picked a learning rate within a good range, your plot look similar Figure 4. If your graph looks very different, especially if your value of J(θ) increases or even blows up, adjust your learning rate and try again. We recommend trying values of the learning rate α on a log-scale, at multiplicative steps of about 3 times the previous value (i.e., 0.3, 0.1, 0.03, 0.01 and so on). You may also want to adjust the number of iterations you are running if that will help you see the overall trend in the curve. To finish this task, just assign a list of candidate learning rates to the variable `alphas` in the following code. You do not need to change any other part of the code.


Notice the changes in the convergence curves as the learning rate changes. With a small learning rate, you should find that gradient descent takes a very long time to converge to the optimal value. Conversely, with a large learning rate, gradient descent might not converge or might even diverge! Using the best learning rate that you found, run gradient descent until convergence to find the final values of θ.

Next, use this value of θ to predict the price of a house with 1650 square feet and 3 bedrooms. You will use value later to check your implementation of the normal equations. Don’t forget to normalize your features when you make this prediction!

In [ ]:
def part2_3():
    
    ### load and prepare data
    X, y = load_data("data/ex1data2.txt")
    
    # normalize features
    X, mu, sigma = feature_normalize(X)

    # add a column of ones to X
    X = np.c_[np.ones((X.shape[0], 1)), X]
    
    ### run gradient descent algorithm for a list of different learning rates
    num_iterations = 400
    
    alphas = [] ### your code here ###

    # for each alpha, try to do gradient descent and plot the convergence curve
    for alpha in alphas:
        theta = np.zeros((3, 1))
        J_history, theta = gradient_descent_multi(X, y, theta, alpha, num_iterations)

        # plot cost versus iteration
        plt.plot(J_history)
        plt.title("Alpha = %f" % (alpha))
        plt.xlabel('Number of iterations')
        plt.ylabel('Cost J')
        plt.xlim([0, 50])
        plt.show()

        ## predict price for a 1650 sq feet 3 bedroom house
        x_test = np.array([1.0, 1650.0, 3.0])
        # exclude intercept units when normalizing
        x_test[1:] = (x_test[1:] - mu) / sigma
        prediction = x_test @ theta
        print('Predicted price = %.2f' % prediction[0])

In [ ]:
part2_3()

## Part 3: Normal Equation

In the lecture videos, you learned that the closed-form solution to linear regression is
$$\theta = (X^T X)^{-1} X^T y$$

Using this formula does not require any feature scaling, and you will get an exact solution in one calculation: there is no “loop until convergence” like in gradient descent. 

Complete the code in `normal_equation()` to use the formula above to calculate θ. Remember that while you don’t need to scale your features, we still need to add a column of 1’s to the X matrix to have an intercept term (θ0).


In [ ]:
from numpy.linalg import pinv

In [ ]:
def normal_equation(X, y):
    raise NotImplementedError('You should implement normal_equation function.')

Now, once you have found θ using this method, use it to make a price prediction for a 1650-square-foot house with 3 bedrooms. You should ﬁnd that gives the same predicted price as the value you obtained using the model fit with gradient descent.

In [ ]:
def part3():
    
    # load and prepare data
    X, y = load_data("data/ex1data2.txt")
    X = np.c_[np.ones((X.shape[0], 1)), X] # add intercept to X

    # find theta using normal equation
    theta = normal_equation(X, y)
    
    # predict price for a 1650 sq feet 3 bedroom house
    x_test = np.array([1.0, 1650.0, 3.0])
    prediction = x_test @ theta
    print('Predicted price = %.2f' % prediction[0])

In [ ]:
part3()

## Submittion

After finishing the assignment, please send **only** this jupyter notebook file via Piazza as a **private** post before the due date. Before submitting your file, be sure to read the following terms and conditions.

**Due date**: November, 2, 2018.

### Terms and Conditions
Please consider the following terms and conditions and take them as serious as you can:
- If you send your code as a public code in Piazza, you will fail in the course.
- If you share all or some parts of your results on the internet or with your friends and classmates in any form, you will fail the course.
- If you submit codes that is not your work, (for example by copying your friend's codes), you will fail the course.
- Any other attempt to cheat, will result in a failure in the course.
- By submmiting this file to Piazza, you confirm that you have read and accepted the above terms and conditions.